In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import os
os.chdir('gdrive/MyDrive/CS753-Project')

In [3]:
!pip install torchaudio

     |████████████████████████████████| 1.9MB 18.7MB/s 


In [11]:
from IPython.display import Audio

## PyTorch things
import torch
import torchaudio
import torch.nn.functional as F

## Other libs
import matplotlib.pyplot as plt
import glob
import os
import random
from tqdm import tqdm_notebook
import torchsummary
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import normalize
import pandas as pd
import seaborn as sn
import librosa
from os import listdir
from pydub import AudioSegment

# feature extractoring and preprocessing data
import librosa
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
from PIL import Image
import pathlib
import csv

# Preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

#Keras
import keras

import warnings
warnings.filterwarnings('ignore')

In [5]:
!pip install pydub

In [ ]:

counter = 0
for f in listdir('genrenew'):
    
    os.mkdir('Wav_data/{}'.format(f)) 
    for f1 in listdir('genrenew/{}'.format(f)):
      counter += 1
      sound = AudioSegment.from_mp3("genrenew/{}/{}".format(f,f1))
      sound.export("Wav_data/{}/{}.wav".format(f,f1[:-4]), format="wav")

print(counter)     

500


In [ ]:
waveform,sample_rate = torchaudio.load("Wav_data/bollypop/bp01.wav")
print(waveform.shape)
print(sample_rate)
if waveform.shape[1] < 128000:
    print('#')
    waveform = F.pad(input=waveform, pad=(0, 128000 - waveform.shape[1]), mode='constant', value=0)
elif waveform.shape[1] > 128000:
    print('#')
    waveform[:,:128] = waveform[:128000]


mfcc =torchaudio.transforms.MFCC(n_mfcc=12, log_mels=True)(waveform)
print(mfcc.size())

torch.Size([2, 1984500])
44100
#
torch.Size([2, 12, 9923])


/usr/local/lib/python3.7/dist-packages/torchaudio/functional/functional.py:358: UserWarning: At least one mel filterbank has all zero values. The value for `n_mels` (128) may be set too high. Or, the value for `n_freqs` (201) may be set too low.
  "At least one mel filterbank has all zero values. "


In [ ]:

y, sr = librosa.load("Wav_data/sufi/su100.wav", mono=True, duration=30)
print(len(y))
mfcc = librosa.feature.mfcc(y=y, sr=sr)
print(mfcc.shape)

661500
(20, 1292)


In [7]:
header = 'filename chroma_stft rmse spectral_centroid spectral_bandwidth rolloff zero_crossing_rate'
for i in range(1, 21):
    header += f' mfcc{i}'
header += ' label'
header = header.split()



In [15]:
data_folder = 'Wav_data'
data_set = []
dictionary_map ={}

file = open('data.csv', 'w', newline='')
with file:
    writer = csv.writer(file)
    writer.writerow(header)

for f in listdir(data_folder):
    for f1 in listdir('Wav_data/{}'.format(f)):
        songname = "Wav_data/{}/{}".format(f,f1)
        y, sr = librosa.load(songname, mono=True, duration=30)
        chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
        spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
        spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
        rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
        zcr = librosa.feature.zero_crossing_rate(y)
        mfcc = librosa.feature.mfcc(y=y, sr=sr)
        to_append = f'{f1} {np.mean(chroma_stft)}  {np.mean(spec_cent)} {np.mean(spec_bw)} {np.mean(rolloff)} {np.mean(zcr)}'    
        for e in mfcc:
            to_append += f' {np.mean(e)}'
        to_append += f' {f}'
        file = open('data.csv', 'a', newline='')
        with file:
            writer = csv.writer(file)
            writer.writerow(to_append.split())

In [18]:
data = pd.read_csv('data.csv')


In [19]:
header = 'filename chroma_stft  spectral_centroid spectral_bandwidth rolloff zero_crossing_rate'.split()
header

['filename',
 'chroma_stft',
 'spectral_centroid',
 'spectral_bandwidth',
 'rolloff',
 'zero_crossing_rate']

In [20]:
ls = ["mfcc{}".format(i) for i in range(1,21)]
col = header + ls + ['label']
col

['filename',
 'chroma_stft',
 'spectral_centroid',
 'spectral_bandwidth',
 'rolloff',
 'zero_crossing_rate',
 'mfcc1',
 'mfcc2',
 'mfcc3',
 'mfcc4',
 'mfcc5',
 'mfcc6',
 'mfcc7',
 'mfcc8',
 'mfcc9',
 'mfcc10',
 'mfcc11',
 'mfcc12',
 'mfcc13',
 'mfcc14',
 'mfcc15',
 'mfcc16',
 'mfcc17',
 'mfcc18',
 'mfcc19',
 'mfcc20',
 'label']

In [22]:
len(col)

27

In [23]:
data.columns = col
data

,filename,chroma_stft,spectral_centroid,spectral_bandwidth,rolloff,zero_crossing_rate,mfcc1,mfcc2,mfcc3,mfcc4,mfcc5,mfcc6,mfcc7,mfcc8,mfcc9,mfcc10,mfcc11,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20,label
0,scl02.wav,0.393630,2793.508825,2724.947491,6103.588092,0.141464,-46.897053,69.555717,-2.016524,23.380373,6.623618,2.421029,-5.757703,5.332809,-4.752931,11.111671,-1.101193,3.130967,1.132255,-1.387890,-5.705366,4.561856,-6.105927,6.502610,-5.407854,-0.274215,semiclassical
1,scl18.wav,0.240682,2767.955528,2433.403858,5631.765945,0.133941,-206.413574,60.938148,-25.842216,-14.850534,-41.712364,7.005772,-34.143299,-8.869285,-9.731552,-11.385138,17.747730,-2.170400,-18.448486,-13.545798,-13.720221,-10.181333,-11.967281,-4.072862,-9.774343,-5.852347,semiclassical
2,scl17.wav,0.363222,2308.646967,2795.509268,5635.874254,0.075055,-76.941551,94.791420,24.020489,17.312582,6.660640,7.705852,5.055718,4.532353,-5.291880,7.381964,-3.674016,6.701849,5.979047,4.467857,3.644095,-0.122741,4.530957,4.428706,2.336652,1.973673,semiclassical
3,scl16.wav,0.342466,2348.082736,2770.920699,5594.391171,0.080740,-111.776947,91.454613,19.832253,15.925972,2.723891,2.694730,8.236013,4.667757,-1.685444,8.099051,-6.971786,2.893494,-0.675740,1.522021,5.271090,2.178024,10.654481,5.887405,2.582098,5.263594,semiclassical
4,scl15.wav,0.331118,2053.418933,2578.628413,4946.695088,0.068787,-145.011810,107.945290,28.677046,19.465176,3.723547,-2.580879,-10.067541,2.661513,-13.122783,12.430797,0.859013,2.073150,5.698961,-3.439064,5.691664,-6.801965,2.424040,-4.921625,-3.753243,-0.622428,semiclassical
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
493,bp93.wav,0.352856,2125.628757,2565.889544,4821.062514,0.069752,-41.297001,98.295143,10.340979,20.613989,9.138204,10.759584,4.724198,2.666525,-4.292716,14.211137,-4.580522,11.360565,-4.500573,4.009302,3.840777,-1.643225,-0.884592,5.073890,-2.874372,4.115224,bollypop
494,bp92.wav,0.288326,1790.175959,2098.668743,3559.620142,0.078040,-82.100998,121.521530,-13.012499,16.843102,0.983627,1.952559,-1.421112,-6.053286,-16.322002,9.392649,-13.231021,4.165483,-9.965364,-3.415564,-1.448866,-6.125137,-2.770978,0.422707,-8.740165,-1.025928,bollypop
495,bp89.wav,0.258287,1970.880200,2250.932461,4033.792275,0.076990,-196.116074,87.885963,3.259030,13.181340,-4.928052,-4.303050,11.983802,-8.931230,-17.724360,7.794697,-10.975510,2.515651,-14.814303,-11.053336,0.575472,-12.467253,-10.170362,2.525185,-22.598131,-6.019463,bollypop
496,bp91.wav,0.277270,2092.021049,2394.208032,4409.115005,0.084209,-97.125534,103.496399,3.592188,12.496719,-4.337034,-2.966832,6.769993,-6.594515,-10.089369,10.556827,-13.134584,2.124217,-14.422090,-0.825191,3.562453,-8.010792,-2.717022,9.178461,-15.040816,-0.868211,bollypop


In [24]:
# Dropping unneccesary columns
data = data.drop(['filename'],axis=1)

In [25]:
genre_list = data.iloc[:, -1]
encoder = LabelEncoder()
y = encoder.fit_transform(genre_list)

In [26]:
scaler = StandardScaler()
X = scaler.fit_transform(np.array(data.iloc[:, :-1], dtype = float))

In [34]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [35]:
from keras import models
from keras import layers

model = models.Sequential()
model.add(layers.Dense(256, activation='relu', input_shape=(X_train.shape[1],)))

model.add(layers.Dense(128, activation='relu'))

model.add(layers.Dense(64, activation='relu'))

model.add(layers.Dense(5, activation='softmax'))

In [36]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [37]:
history = model.fit(X_train,
                    y_train,
                    epochs=20,
                    batch_size=128)

Epoch 1/20
4/4 [==============================] - 0s 4ms/step - loss: 1.6261 - accuracy: 0.2031
Epoch 2/20
4/4 [==============================] - 0s 3ms/step - loss: 1.4225 - accuracy: 0.4699
Epoch 3/20
4/4 [==============================] - 0s 3ms/step - loss: 1.2865 - accuracy: 0.5819
Epoch 4/20
4/4 [==============================] - 0s 3ms/step - loss: 1.1512 - accuracy: 0.6893
Epoch 5/20
4/4 [==============================] - 0s 3ms/step - loss: 1.0305 - accuracy: 0.7424
Epoch 6/20
4/4 [==============================] - 0s 3ms/step - loss: 0.9439 - accuracy: 0.7095
Epoch 7/20
4/4 [==============================] - 0s 3ms/step - loss: 0.8503 - accuracy: 0.7315
Epoch 8/20
4/4 [==============================] - 0s 3ms/step - loss: 0.7721 - accuracy: 0.7368
Epoch 9/20
4/4 [==============================] - 0s 3ms/step - loss: 0.6976 - accuracy: 0.7438
Epoch 10/20
4/4 [==============================] - 0s 3ms/step - loss: 0.6589 - accuracy: 0.7733
Epoch 11/20
4/4 [======================

In [38]:
test_loss, test_acc = model.evaluate(X_test,y_test)

4/4 [==============================] - 0s 3ms/step - loss: 0.6265 - accuracy: 0.8000


In [39]:
print('test_acc: ',test_acc*100)

test_acc:  80.0000011920929


In [ ]:
data_folder = 'Wav_data'
data_set = []
dictionary_map ={}
count = 0
for f in listdir(data_folder):
    if f not in dictionary_map:
      print(f)
      dictionary_map[f] = count
      count +=1
    for f1 in listdir('Wav_data/{}'.format(f)):
      # print(f1)
    #   waveform,sample_rate = torchaudio.load("Wav_data/{}/{}".format(f,f1))
      y, sr = librosa.load("Wav_data/{}/{}".format(f,f1))
      mfcc = librosa.feature.mfcc(y=y, sr=sr)
    #   print(waveform.shape[1])
    #   if waveform.shape[1] < 128000:
    #     waveform = F.pad(input=waveform, pad=(0, 128000 - waveform.shape[1]), mode='constant', value=0)
    #   elif waveform.shape[1] > 128000:
    #     waveform = waveform[:128000]
    #   print(waveform.shape[1])
    #   mfcc = torchaudio.transforms.MFCC(n_mfcc=12, log_mels=True)(waveform).squeeze(0).transpose(0,1)
    #   print(mfcc.size())
    #   print(dictionary_map[f])
      data_set.append([mfcc,dictionary_map[f]])
# print(data_set)


semiclassical
carnatic
sufi
ghazal
bollypop


In [ ]:
len(data_set)

500

In [ ]:
import random
random.shuffle(data_set)
train_set = data_set[:400]
test_set = data_set[-100:]

train_dl = torch.utils.data.DataLoader(train_set, batch_size=16, shuffle=True)
test_dl = torch.utils.data.DataLoader(test_set, batch_size=16)

In [ ]:
num = 0
for i in data_set:
    num = max(i[0].shape[1],num)
print(num)

4522


In [ ]:
class SpeechRNN(torch.nn.Module):
  
  def __init__(self):
    super(SpeechRNN, self).__init__()
    
    self.rnn = torch.nn.LSTM(input_size = 1938, 
                              hidden_size= 256, 
                              num_layers = 2, 
                              batch_first=True)
    
    self.out_layer = torch.nn.Linear(256, 5)
    
    self.softmax = torch.nn.LogSoftmax(dim=1)
    
  def forward(self, x):
    
    out, _ = self.rnn(x)
    
    x = self.out_layer(out[:,-1,:])
    
    return self.softmax(x)

In [ ]:
##RE-RUN THIS CODE TO GET A "NEW" NETWORK

LEARNING_RATE = 0.001

## Create an instance of our network
net = SpeechRNN()

## Move it to the GPU
# net = net.cuda()

# Negative log likelihood loss
criterion = torch.nn.NLLLoss()

# Adam optimizer
optimizer = torch.optim.Adam(net.parameters(), lr=LEARNING_RATE)

In [ ]:
## NUMBER OF EPOCHS TO TRAIN
N_EPOCHS = 20

epoch_loss, epoch_acc, epoch_val_loss, epoch_val_acc = [], [], [], []

for e in range(N_EPOCHS):
  
  print("EPOCH:",e)
  
  ### TRAINING LOOP
  running_loss = 0
  running_accuracy = 0
  
  ## Put the network in training mode
  net.train()
  
  for i, batch in enumerate(tqdm_notebook(train_dl)):
    
    # Get a batch from the dataloader
    x = batch[0]
    labels = batch[1]
    
    # move the batch to GPU
    x = x
    labels = labels

    # Compute the network output
    y = net(x)
    # print(y.shape)
    # print(min(labels),max(labels))
    # Compute the loss
    loss = criterion(y, labels)
    
    # Reset the gradients
    optimizer.zero_grad()
    
    # Compute the gradients
    loss.backward()
    
    # Apply one step of the descent algorithm to update the weights
    optimizer.step()
    
    ## Compute some statistics
    with torch.no_grad():
      running_loss += loss.item()
      running_accuracy += (y.max(1)[1] == labels).sum().item()
    
  print("Training accuracy:", running_accuracy/float(len(train_set)),
        "Training loss:", running_loss/float(len(train_set)))
  
  epoch_loss.append(running_loss/len(train_set))
  epoch_acc.append(running_accuracy/len(train_set))
  
  

EPOCH: 0


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


RuntimeError: ignored